In [4]:
import pandas as pd 
import tensorflow as tf 
from tensorflow import keras 
import numpy as np 
import matplotlib.pyplot as plt 

In [5]:
df = pd.read_csv("/Users/parminder/Documents/MachineLearning/datasets/insurance_data.csv")
df.head()


age  affordibility  bought_insurance
0   22              1                 0
1   25              0                 0
2   47              1                 1
3   52              0                 0
4   46              1                 1

In [6]:
#Split Train and Test Data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['age', 'affordibility']],df.bought_insurance, test_size=0.2, random_state=25)

In [7]:
#Preprocessing: Scale the data so that both age and affordibility are in same scaling range
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age']/100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age']/100

In [ ]:
#MODEL BUILDING- 1) Using keras Tensor Flow
#2) Using python scratch
# Compare the results obtained in 1 and 2 

In [ ]:
#1) First build a model in keras/tensorflow and see what weights and bias values it comes up with. We will than try to reproduce same weights and bias in our plain python implementation of gradient descent. Below is the architecture of our simple neural network

In [8]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,), activation= 'sigmoid', kernel_initializer= 'ones', bias_initializer = 'zeros')
])

model.compile(optimizer='adam',
loss='binary_crossentropy',
metrics=['accuracy']) 

model.fit(X_train_scaled, y_train, epochs=5000)

=========] - 0s 7ms/step - loss: 0.4669 - accuracy: 0.9091
Epoch 4804/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.4669 - accuracy: 0.9091
Epoch 4805/5000
1/1 [==============================] - 0s 6ms/step - loss: 0.4669 - accuracy: 0.9091
Epoch 4806/5000
1/1 [==============================] - 0s 5ms/step - loss: 0.4668 - accuracy: 0.9091
Epoch 4807/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.4668 - accuracy: 0.9091
Epoch 4808/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.4668 - accuracy: 0.9091
Epoch 4809/5000
1/1 [==============================] - 0s 10ms/step - loss: 0.4668 - accuracy: 0.9091
Epoch 4810/5000
1/1 [==============================] - 0s 7ms/step - loss: 0.4668 - accuracy: 0.9091
Epoch 4811/5000
1/1 [==============================] - 0s 10ms/step - loss: 0.4667 - accuracy: 0.9091
Epoch 4812/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.4667 - accuracy: 0.9091
Epoch 4813/5000
1/1 [=========

In [9]:
model.evaluate(X_test_scaled, y_test) #gave 1 means 100 percent accuracy

1/1 [==============================] - 0s 379ms/step - loss: 0.3550 - accuracy: 1.0000


[0.35497748851776123, 1.0]

In [10]:
model.predict(X_test_scaled) #>0.5 - will buy insurance, if < 0.5- person wont buy insurance

array([[0.7054848 ],
       [0.35569546],
       [0.16827846],
       [0.47801173],
       [0.7260697 ],
       [0.8294984 ]], dtype=float32)

In [11]:
y_test

2     1
10    0
21    0
11    0
14    1
9     1
Name: bought_insurance, dtype: int64

In [12]:
#Now get the value of weights and bias from the model: coef , intercept is known w1, w2, bias
coef , intercept = model.get_weights()

In [13]:
coef, intercept

#this means that w1 = 5.060867 , w2 = 1.4086502, bias = 2.9137027

(array([[5.0608673],
        [1.4086503]], dtype=float32),
 array([-2.913703], dtype=float32))

In [ ]:
#now without using tensor , doing the above with plain python, 
# Step 1: First define sigmoid function 
# y= 1 / (1+ e (power -x)
#then 

In [14]:
def sigmoid(x):
    import math 
    return 1 / (1+math.exp(-x))
sigmoid(18)

0.9999999847700205

In [15]:
X_test

age  affordibility
2    47              1
10   18              1
21   26              0
11   28              1
14   49              1
9    61              1

In [ ]:
#Step 2: Form Prediction function with below formula
#Instead of model.predict, write our own prediction function that uses w1,w2 and bias
#w1*age + w2*affordibility + bias
#w1= coef[0] , w2 = coef[1], bias = intercept

In [16]:
def prediction_function(age, affordibility):
    weighted_sum = coef[0] * age + coef[1] * affordibility + intercept
    return sigmoid(weighted_sum)

prediction_function(.47, 1) #this is same as model.predict(X_test_scaled) 
#this shows how prediction function workd once we have weights & bias. 

0.705484819775958

In [17]:
prediction_function(.18, 1) #this is same as model.predict(X_test_scaled) 

0.35569544317951163

In [ ]:
#after the above prediction using Sigmoid. Now we do it using Gradient Descent

#Now we start implementing gradient descent in plain python. Again the goal is to come up with same w1, w2 and bias that keras model calculated. We want to show how keras/tensorflow would have computed these values internally using gradient descent

In [18]:
#Mean Abs error or mae or called as COST FUNCTION = 1/n Sum [abs(yi-y^)]
#indiviual errors is called LOSS i.e Total Error = err1+err2+...+err13 (since we have 13 training samples that was sent to neuron)
#epoch means Going thru all training samples ONCE.
#For MAE use loss = 'mean_absolute_error' 
#For Mean Squared Error MSE = 1/n Sum [abs(yi-y^)power2]. use loss = 'mean_absolute_error' 

#For Log Loss or Binary Cross Entropy = -1/n Sum [yi log(y^)+ (1-yi). log(1-y^)] loss = 'binary_cross_entropy'

#PS: For Logistic Regression MSE should not be used.   Log Loss or Binary Cross Entropy is used. https://towardsdatascience.com/why-not-mse-as-a-loss-function-for-logistic-regression-589816b5e03c 


def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i, epsilon) for i in y_predicted]
    y_predicted_new = [min(i, 1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true * np.log(y_predicted_new) + (1-y_true)* np.log(1-y_predicted_new))





In [19]:
def sigmoid_numpy(X):
    return 1/(1+np.exp(-X))

sigmoid_numpy(np.array([12, 0 , 1]))

array([0.99999386, 0.5       , 0.73105858])

In [20]:
#gradient_descent func helps to find out weights i.e w1, w2, bias 

# w1 = w1 - learning rate *der/der W1          der/der W1  = 1/n SUM[xi (y^-yi)].      here mean is done using dot product and transpose
# b = b - learning rate * der/der B             der/der b = 1/n SUM[(yi^ - yi)]
def gradient_descent(age, affordibility, y_true, epochs, loss_threshold):
    w1 = w2 = 1
    bias = 0
    rate = 0.5 
    n = len(age)

    for i in range(epochs):
        weighted_sum =  w1 * age + w2 * affordibility + bias 
        y_predicted = sigmoid_numpy(weighted_sum)
        
        loss = log_loss(y_true, y_predicted)

        w1d= (1/n) * np.dot(np.transpose(age) , (y_predicted - y_true))

        w2d = (1/n) * np.dot(np.transpose(affordibility) , (y_predicted - y_true))
        bias_d = np.mean(y_predicted - y_true)

        w1 = w1 - rate * w1d
        w2 = w2 - rate * w2d
        bias = bias - rate * bias_d

        print(f'Epoch:{i}, w1: {w1}, w2: {w2}, bias:{bias}, loss:{loss}')

        if(loss<=loss_threshold):
            break
        
    return w1, w2, bias


In [89]:
#how to call the gradient descent function? passing the parameter
#age - its a vector AGE , not single age value : so we numpy 
# 
gradient_descent(X_train_scaled['age'], X_train_scaled['affordibility'], y_train, 5000, 0.4631)


37484761812156, bias:-2.158892638895851, loss:0.5065955884569808
Epoch:182, w1: 3.235768256672519, w2: 1.3943636051358148, bias:-2.1641946744139777, loss:0.5062627148655543
Epoch:183, w1: 3.2474690264778157, w2: 1.394970782361845, bias:-2.1694814133009186, loss:0.505931227333758
Epoch:184, w1: 3.2591476781228836, w2: 1.3955701697511878, bias:-2.1747529887145927, loss:0.5056011184990193
Epoch:185, w1: 3.270804232829102, w2: 1.3961619261439753, bias:-2.1800095317363692, loss:0.505272381068909
Epoch:186, w1: 3.2824387123797556, w2: 1.3967462073858943, bias:-2.185251171408572, loss:0.5049450078195657
Epoch:187, w1: 3.294051139107937, w2: 1.3973231663843697, bias:-2.1904780347712607, loss:0.5046189915941715
Epoch:188, w1: 3.3056415358846545, w2: 1.3978929531636493, bias:-2.1956902468983084, loss:0.5042943253014793
Epoch:189, w1: 3.317209926107157, w2: 1.3984557149188142, bias:-2.2008879309327845, loss:0.5039710019143897
Epoch:190, w1: 3.3287563336874557, w2: 1.3990115960687382, bias:-2.2060

(5.051047623653049, 1.4569794548473887, -2.9596534546250037)

In [22]:
coef, intercept # and the results totally match

(array([[5.0608673],
        [1.4086503]], dtype=float32),
 array([-2.913703], dtype=float32))

In [ ]:
#This shows that in the end we were able to come up with same value of w1,w2 and bias using a plain python implementation of gradient descent function

In [ ]:
#Implementing with the help of CLASS

In [41]:
class myNN:
    def __init__(self):
        self.w1 = 1
        self.w2 = 1
        self.bias = 0

    def fit(self, X, y, epochs, loss_threshold):
        self.w1, self.w2, self.bias = self.gradient_descent(X['age'], X['affordibility'], y, epochs, loss_threshold)
        print(f"Final weights and bias: w1: {self.w1}, w2: {self.w2}, bias: {self.bias}")

    def predict(self, X_test):
        weighted_sum = self.w1*X_test['age'] + self.w2*X_test['affordibility'] + self.bias
        return sigmoid_numpy(weighted_sum)

    def gradient_descent(self, age,affordability, y_true, epochs, loss_thresold):
        w1 = w2 = 1
        bias = 0
        rate = 0.5
        n = len(age)
        for i in range(epochs):
            weighted_sum = w1 * age + w2 * affordability + bias
            y_predicted = sigmoid_numpy(weighted_sum)
            loss = log_loss(y_true, y_predicted)
            
            w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true)) 
            w2d = (1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true)) 

            bias_d = np.mean(y_predicted-y_true)
            w1 = w1 - rate * w1d
            w2 = w2 - rate * w2d
            bias = bias - rate * bias_d
            
            if i%50==0:
                print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
            
            if loss<=loss_thresold:
                print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
                break

        return w1, w2, bias  

In [42]:
customModel = myNN()
customModel.fit(X_train_scaled, y_train, epochs= 100, loss_threshold=0.4631)

Epoch:0, w1:0.974907633470177, w2:0.948348125394529, bias:-0.11341867736368583, loss:0.7113403233723417
Epoch:50, w1:1.503319554173139, w2:1.108384790367645, bias:-1.2319047301235464, loss:0.5675865113475955
Final weights and bias: w1: 2.1871944491052364, w2: 1.2918774898345378, bias: -1.6533819823304428


In [32]:
coef, intercept
#This shows that in the end we were able to come up with same value of w1,w2 and bias using a plain python implementation of gradient descent function

(array([[5.0608673],
        [1.4086503]], dtype=float32),
 array([-2.913703], dtype=float32))

In [33]:
X_test_scaled

age  affordibility
2   0.47              1
10  0.18              1
21  0.26              0
11  0.28              1
14  0.49              1
9   0.61              1

In [34]:
#1) Predict using custom model
customModel.predict(X_test_scaled)

2     0.695713
10    0.636006
21    0.451655
11    0.657189
14    0.699625
9     0.722476
dtype: float64

In [35]:
#(2) Predict using tensorflow model
model.predict(X_test_scaled)
#Above you can compare predictions from our own custom model and tensoflow model. You will notice that predictions are almost same

array([[0.7054848 ],
       [0.35569546],
       [0.16827846],
       [0.47801173],
       [0.7260697 ],
       [0.8294984 ]], dtype=float32)

In [ ]:
#https://github.com/codebasics/deep-learning-keras-tf-tutorial/blob/master/7_nn_from_scratch/7_neural_network_from_scratch.ipynb